## Implementing search

We have to implement the search now, in order to do so we have to keep some things in mind:

- From each inverted index, there will be a row of doc_ids received, for each doc_id, we need to gather the document information we have to display
- We also need to employ a ranking algorithm, that can process both single word and multiple word queries
- 

In [1]:
from searching import search
from docdata.getdocdata.getjson import get_docdata

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### We will start by checking if we can retrieve the docdata from its doc_id

In [2]:
%%time
get_docdata(21321)

CPU times: user 4.19 ms, sys: 247 µs, total: 4.44 ms
Wall time: 3.47 ms


{'url': 'https://www.thesun.co.uk/sport/18399667/tobin-heath-released-arsenal-contract/',
 'title': 'Tobin Heath’s Arsenal spell ends with USA ace released early from contract by mutual consent following injury',
 'date': '2022-04-28',
 'chars500': 'TOBIN HEATH ’ s Arsenal stint has come to an end with the forward leaving the club by mutual agreement following a hamstring injury .\nThe USA forward has been contending with a minor problem which has ruled her out of the Gunners ’ last three games of the season .\nHeath ’ s one-year contract was due to expire at the end of the 2021-22 term .\nHowever , hamstring issues have put the player , 33 , out of contention for Arsenal ’ s games against Aston Villa , Tottenham and West Ham .\nThe club are ta',
 'author': 'Sandra Brobbey'}

### Implemented this, now we will proceed on getting the doc ids we want to display, but first we need to gather the word ids from the query string, so we need to process that

In [3]:
from searching.search import conv_query_wordids, get_wordelems, return_common_docelems

In [4]:
conv_query_wordids("birthday brother special doing")

[128025, 94298, 139152]

In [5]:
get_wordelems("birthday brother")

[[<invertedindex.invDS.DocElement at 0x7f8a37bec040>,
  ...],
  ...]]

### Now we will try to get an inverted index word list from the word id we give it

In [6]:
from invertedindex.gettingWordLists import get_list, get_lists

In [7]:
get_list(893)[0].doc_id

47570

In [8]:
%%time
get_lists(conv_query_wordids("super brother inside doing"))

CPU times: user 467 ms, sys: 32.5 ms, total: 499 ms
Wall time: 485 ms


[[<invertedindex.invDS.DocElement at 0x7f8a2d56eb30>,
  ...],
  ...],
  ...]]

### Now we are able to access the list of docs from the query, time to implement the multi word and the single word search query

### Implementing this for our docelements now

In [9]:
return_common_docelems(get_lists(conv_query_wordids("sister brother hard doing")))[0]

### Implementing a function to return rank now

We will start by calculating based on position

In [10]:
import heapq

def merge_sorted_arrays(arrays):
    result = []
    min_heap = []

    # Initialize the min heap with the first element from each array
    for i, array in enumerate(arrays):
        if array:  # Ensure the array is not empty
            heapq.heappush(min_heap, (array[0], i, 0))

    while min_heap:
        val, array_index, position = heapq.heappop(min_heap)
        result.append(val)

        # Move to the next position in the current array
        position += 1

        # If there are more elements in the current array, push the next one into the heap
        if position < len(arrays[array_index]):
            heapq.heappush(min_heap, (arrays[array_index][position], array_index, position))

    return result

# Example usage:
arrays = [
    [1, 3, 5],
    [2, 4, 6],
    [0, 7, 8]
]

sorted_result = merge_sorted_arrays(arrays)
print(sorted_result)


[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [11]:
# Given arrays of positions, we need to give a proximity score

positions1 = [
    [5, 10, 15, 20],
    [8, 13, 18, 23],
    [30, 35, 40, 45]
]

positions2 = [
    [5, 10, 15, 20],
    [6, 11, 16, 21],
    [7, 12, 17, 22]
]

    

def proximity_score(positions):
    # We will start by merging these
    merged_pos = merge_sorted_arrays(positions)
    differences = []
    for i in range(1,len(merged_pos)):
        differences.append(merged_pos[i]-merged_pos[i-1])

    difference_score = sum(differences)/len(differences)
    
    print(merged_pos)
    print(difference_score)
    return 100/difference_score

print(proximity_score(positions1))
print(proximity_score(positions2))


[5, 8, 10, 13, 15, 18, 20, 23, 30, 35, 40, 45]
3.6363636363636362
27.5
[5, 6, 7, 10, 11, 12, 15, 16, 17, 20, 21, 22]
1.5454545454545454
64.70588235294117


#### We will check other ranking now

In [12]:
from searching.ranking.rank import return_rank

In [13]:
ranked_results = [(-return_rank(each).importance, return_rank(each).doc_id) for each in return_common_docelems(get_lists(conv_query_wordids("brother a monster")))]

[415]
1000.0
[415]
1000.0
[219]
1000.0
[219]
1000.0
[1355, 1792]
437.5624375624376
[1355, 1792]
437.5624375624376
[1256]
1000.0
[1256]
1000.0
[5, 16]
11.988011988011989
[5, 16]
11.988011988011989
[235]
1000.0
[235]
1000.0
[337]
1000.0
[337]
1000.0
[893]
1000.0
[893]
1000.0
[1052]
1000.0
[1052]
1000.0
[357]
1000.0
[357]
1000.0
[162]
1000.0
[162]
1000.0
[141]
1000.0
[141]
1000.0
[400]
1000.0
[400]
1000.0
[18, 41]
23.976023976023978
[18, 41]
23.976023976023978
[140]
1000.0
[140]
1000.0
[227]
1000.0
[227]
1000.0
[131]
1000.0
[131]
1000.0
[159]
1000.0
[159]
1000.0
[134]
1000.0
[134]
1000.0
[12, 39, 140]
64.46776611694153
[12, 39, 140]
64.46776611694153
[171]
1000.0
[171]
1000.0
[108]
1000.0
[108]
1000.0
[532]
1000.0
[532]
1000.0
[182, 353, 360]
89.4552723638181
[182, 353, 360]
89.4552723638181
[376]
1000.0
[376]
1000.0
[218]
1000.0
[218]
1000.0
[145]
1000.0
[145]
1000.0
[4]
1000.0
[4]
1000.0
[98]
1000.0
[98]
1000.0
[440, 451]
11.988011988011989
[440, 451]
11.988011988011989
[75, 553]
478.52

In [14]:
ranked_results[:30]

[(-11.05, 13),
 (-11.05, 68),
 (-102.1142694063927, 99),
 (-11.05, 106),
 (-20.170833333333334, 117),
 (-11.05, 195),
 (-11.05, 197),
 (-11.05, 200),
 (-11.05, 200),
 (-11.05, 259),
 (-11.05, 261),
 (-11.05, 265),
 (-11.05, 265),
 (-18.085416666666667, 286),
 (-11.05, 290),
 (-11.05, 290),
 (-11.05, 296),
 (-11.05, 299),
 (-11.05, 301),
 (-19.775581395348837, 327),
 (-11.05, 346),
 (-11.05, 421),
 (-11.05, 448),
 (-13.558938547486033, 453),
 (-11.05, 506),
 (-11.05, 515),
 (-11.05, 517),
 (-13.05, 526),
 (-11.05, 526),
 (-16.170833333333334, 566)]

In [15]:
heapq.heapify(ranked_results)

In [16]:
ranked_results[:30]

[(-148.45512742718446, 23173),
 (-113.98546798029557, 36370),
 (-140.71987818383167, 23486),
 (-109.60248493975904, 39086),
 (-113.26567065073041, 5699),
 (-105.30264750378215, 48845),
 (-113.27784722222222, 15064),
 (-104.86235632183909, 87677),
 (-104.36268115942029, 5023),
 (-105.33666666666667, 118839),
 (-110.49685430463576, 10713),
 (-102.5159793814433, 49758),
 (-104.02496672212979, 12150),
 (-108.09347129506008, 7177),
 (-103.28333333333333, 14950),
 (-35.06398176291793, 8248),
 (-104.6179012345679, 16049),
 (-103.38929961089494, 103583),
 (-103.29169096209912, 17954),
 (-103.43689956331878, 117353),
 (-104.46027607361964, 42933),
 (-108.41796657381616, 21058),
 (-40.35577075098814, 11045),
 (-102.23064516129033, 23404),
 (-102.1454941860465, 11852),
 (-103.01619980569949, 12149),
 (-102.42415254237288, 62106),
 (-104.9039156626506, 13060),
 (-103.03082255083179, 14171),
 (-31.967903225806452, 28438)]

In [25]:

get_docdata(23173)

{'url': 'https://www.thesun.co.uk/tvandshowbiz/18501194/will-young-threw-brother-out-suicide/',
 'title': 'Alcohol made my brother a monster and I threw him out a week before his suicide – I couldn’t save him, says Will Young',
 'date': '2022-05-08',
 'chars500': 'BORN just minutes apart , singer Will Young and his twin brother Rupert shared an unbreakable bond .\nThey looked and sounded alike — but Will became a household name after winning Pop Idol in 2002 , while Rupert once joked he only had the nerve to sing “ after a few beers ” .\nAnd their lives took completely different paths .\nWill is an award-winning chart star but his brother Rupert tragically took his own life nearly two years ago after a long battle with depression and alcohol addiction .\nNow ',
 'author': 'Timothy Sigsworth'}

In [17]:
get_docdata(23486)

{'url': 'https://www.thesun.co.uk/tv/18527163/will-young-losing-my-twin-tears-rupert-death/',
 'title': 'Will Young: Losing My Twin viewers in floods of tears as star opens up about tragic death of brother for first time',
 'date': '2022-05-10',
 'chars500': "WILL Young left viewers in floods of tears as he opened up about his twin brother 's tragic death in a new documentary .\nSinger Will is an award-winning chart star but his brother Rupert tragically took his own life nearly two years ago after a long battle with depression and alcohol addiction .\nWill , 43 , spoke out for the first time since the death of his twin in a Channel 4 documentary Will Young : Losing My Twin Rupert .\nHe tells how a previous suicide attempt left Rupert watching him perfo",
 'author': 'Shan Ally'}

### For starters, this work, now we need to add everything up on our scripts

### Taking it one step at a time now

First of all, there will be a query string

In [1]:
query_string = "corona virus striked again"

After that it will be used to get the wordelems from the inverted index

In [2]:
from searching.search import get_wordelems, return_common_docelems

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
%%time
all_wordelems = get_wordelems(query_string)

CPU times: user 1.08 s, sys: 33.1 ms, total: 1.11 s
Wall time: 1.11 s


After that we will be getting the common docelems

In [4]:
%%time
common_doc_elems = return_common_docelems(all_wordelems)


CPU times: user 9.8 ms, sys: 0 ns, total: 9.8 ms
Wall time: 9.66 ms


Now we will be ranking each of these

In [5]:
from searching.ranking.rank import return_sorted_docs

In [6]:
%%time
ourresults = return_sorted_docs(common_doc_elems)

[258]
1000.0
[344]
1000.0
[55]
1000.0
[273]
1000.0
[690]
1000.0
[3913]
1000.0
[131]
1000.0
[56]
1000.0
[365]
1000.0
[44]
1000.0
[12]
1000.0
[154]
1000.0
[664, 712]
48.95104895104895
[330, 350]
20.97902097902098
[265]
1000.0
[162]
1000.0
[1546, 1612]
66.93306693306694
[375]
1000.0
[332]
1000.0
[906]
1000.0
[457]
1000.0
[5, 344]
339.6603396603397
[146, 195]
49.95004995004996
[526]
1000.0
[638]
1000.0
[314, 630]
316.6833166833167
[145, 251, 356]
105.94702648675663
[405]
1000.0
[714]
1000.0
[277, 326]
49.95004995004996
[410]
1000.0
[59, 82, 915]
428.2858570714643
[110]
1000.0
[332, 440]
108.8911088911089
[222]
1000.0
[334]
1000.0
[188]
1000.0
[107]
1000.0
[175, 194, 253, 317, 654, 1038]
172.76544691061787
[392, 400]
8.991008991008991
[1077]
1000.0
[337]
1000.0
[329]
1000.0
[329, 452]
123.87612387612388
[45, 250]
205.79420579420582
[537]
1000.0
[39, 152, 269]
115.44227886056971
[162, 457]
295.70429570429576
[59]
1000.0
[505]
1000.0
[693]
1000.0
[11]
1000.0
[1]
1000.0
[18, 53, 252, 319]
100.

In [7]:
ourresults[:30]

[(-127.02499999999999, 3064),
 (-127.02499999999999, 8424),
 (-113.46415313225059, 5434),
 (-111.51711478800414, 121529),
 (-108.5683712121212, 13157),
 (-107.83812849162011, 100226),
 (-107.72651331719128, 86566),
 (-107.59910179640718, 115546),
 (-107.54772727272727, 5440),
 (-107.37674199623352, 7232),
 (-107.22947247706422, 27478),
 (-106.58422897196262, 101965),
 (-105.87, 42062),
 (-105.4311422413793, 14733),
 (-105.3617540687161, 123765),
 (-105.31265625, 52064),
 (-104.83361111111111, 81666),
 (-104.391015625, 18934),
 (-104.36069711538461, 58910),
 (-104.33145695364239, 112168),
 (-104.31644736842105, 124977),
 (-104.30895061728395, 52751),
 (-104.29615384615384, 52130),
 (-104.2349765258216, 51938),
 (-104.22462574850299, 123306),
 (-104.22262611275964, 122180),
 (-104.20428571428572, 84356),
 (-104.18958333333333, 34362),
 (-104.17266973532796, 100267),
 (-104.11453089244851, 101829)]

### Checking our final function

In [1]:
from searching.search import get_results

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
%%time
len(get_results("ireland prime minister"))

CPU times: user 98.2 ms, sys: 8.38 ms, total: 107 ms
Wall time: 105 ms


14443

In [13]:
return_rank(return_common_docelems(get_lists(conv_query_wordids("sister brother hard doing")))[0])

In [23]:
# def find_common_elements(arrays):
#     if not arrays or any(not array for array in arrays):
#         return []

#     result = []
#     pointers = [0] * len(arrays)

#     while 1:

#         # We will start by generating a vertical list so that we can find the minimum, this is done in O(h) time where h = no.of words
#         current_list = [arrays[i][pointers[i]] if pointers[i] < len(arrays[i]) else 1000000 for i in range(len(arrays))]

#         # We find the min, also done in O(h)
#         min_val = min(current_list)

#         if min_val >= 1000000:
#             return result
        

#         # Min indexes, found in O(h) as well
#         min_indexes = [i for i in range(len(arrays)) if current_list[i]==min_val]

#         # We relax the min indexes by appending their values to the results, also done in O(h)
#         result.append([arrays[i][pointers[i]] for i in min_indexes])
#         # We also increment all the min indexes
#         for i in min_indexes:
#             pointers[i] += 1

#     return result

# # Example usage:
# arrays = [
#     [1, 3, 4, 6, 10],
#     [3, 4, 6, 11, 13, 17],
#     [5, 8, 14, 15, 17],
#     # Add more arrays as needed
# ]

# result = find_common_elements(arrays)
# print(result)


[[1], [3, 3], [4, 4], [5], [6, 6], [8], [10], [11], [13], [14], [15], [17, 17]]


In [45]:
# def return_common_docelems(arrays):
#     if not arrays or any(not array for array in arrays):
#         return []

#     result = []
#     pointers = [0] * len(arrays)

#     while 1:

#         # We will start by generating a vertical list so that we can find the minimum, this is done in O(h) time where h = no.of words
        
#         current_list = [arrays[i][pointers[i]].doc_id if pointers[i] < len(arrays[i]) else 1000000 for i in range(len(arrays))]

#         # We find the min, also done in O(h)
#         min_val = min(current_list)

#         if min_val >= 1000000:
#             return result
        

#         # Min indexes, found in O(h) as well
#         min_indexes = [i for i in range(len(arrays)) if current_list[i]==min_val]

#         # We relax the min indexes by appending their values to the results, also done in O(h)
#         result.append([arrays[i][pointers[i]] for i in min_indexes])
#         # We also increment all the min indexes
#         for i in min_indexes:
#             pointers[i] += 1

#     return result


In [33]:
a = [3, 5]
b = [5, 6]
c = [[5, 6], [4, 3]]
[each for each in c]

[[5, 6], [4, 3]]